In [2]:
# import sys
# sys.path.append('/media/')
# from helperToolz.helpsters import *
# from helperToolz.dicts_and_lists import *

import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.dicts_and_lists import *
from helperToolz.guzinski import *
from skimage import measure
from skimage.measure import regionprops

In [3]:
year = 2023
model_name = 'AI4_RGB_exclude_False_47'
ncores = 56


pred_list = [file for file in getFilelist(f'/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/{model_name}/{year}/vrt/', '.vrt')\
              if 'unmasked' not in file]
ref_list = [file for file in getFilelist(f'/data/Aldhani/eoagritwin/fields/IACS/4_Crop_mask/{year}/', '.tif') if 'prediction_extent' in file]

pred_list_sorted, ref_list_sorted = [], []
for pred in pred_list:
    for ref in ref_list:
        if '_'.join(pred.split('/')[-1].split('_')[:-2]) == ref.split('cropMask_')[-1].split('_prediction_extent')[0]:
            pred_list_sorted.append(pred)
            ref_list_sorted.append(ref)
        else:
            pass

In [ ]:
year = 2023

pred_list = [file for file in getFilelist(f'/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/{year}/vrt/', '.vrt') if 'unmasked' not in file]
ref_list = [file for file in getFilelist(f'/data/Aldhani/eoagritwin/fields/IACS/4_Crop_mask/{year}/', '.tif') if 'prediction_extent' in file]

pred_list_sorted, ref_list_sorted = [], []
for pred in pred_list:
    for ref in ref_list:
        if '_'.join(pred.split('/')[-1].split('_')[:-2]) == ref.split('cropMask_')[-1].split('_prediction_extent')[0]:
            pred_list_sorted.append(pred)
            ref_list_sorted.append(ref)
        else:
            pass

In [7]:
prediction = pred_list_sorted[0]
reference = ref_list_sorted[0]
reference.split('/')[-1].split('.')[0]

'GSA-DE_BRB-2023_cropMask_lines_touch_false_crop_touch_false_prediction_extent'

In [ ]:
for prediction, reference in zip(pred_list_sorted, ref_list_sorted):

    # set the number by which rows and cols will be divided --> determines the number of tiles // also set border limit (dont sample fields too close to tile borders) and sample size
    slicer = 10
    border_limit = 5
    sample_size  = 10000
    # set the number of cores for parallel processing and set seed
    ncores = 55
    np.random.seed(42)
    make_tifs_from_intermediate_step = True

    ######### prepare job-list

    # create lists that will be passed on to the joblist
    # tile_list = []
    extent_true_list = []
    extent_pred_list = []
    boundary_pred_list = []
    result_dir_list = []
    row_col_start = []


    # tile predictions in prds --> total extent encompasses 90 Force Tiles (+ a few rows and cols that will be neglected as they are outside of study area)
    pred_ds = gdal.Open(prediction)
    rows, cols = pred_ds.RasterYSize, pred_ds.RasterXSize

    row_start = [i for i in range(0, rows, math.floor(rows/slicer))]
    row_end = [i for i in range (math.floor(rows/slicer), rows, math.floor(rows/slicer))]
    row_start = row_start[:len(row_end)] 

    col_start = [i for i in range(0, cols, math.floor(cols/slicer))]
    col_end = [i for i in range (math.floor(cols/slicer), cols, math.floor(cols/slicer))]
    col_start = col_start[:len(col_end)] 

        # load IACS reference mask and label it 
    ref_ds = gdal.Open(reference)
    extent_true = ref_ds.GetRasterBand(1).ReadAsArray()
    binary_true = extent_true > 0
    instances_true = measure.label(binary_true, background=0, connectivity=1)

    # sample fields
    # build a mask to exclude fields that are in border_limit to tile borders
    power_mask = np.zeros(instances_true.shape)
    for i in range(len(row_end)):
        for j in range(len(col_end)):
                power_mask[row_start[i]:row_start[i] + border_limit, :] = 1
                power_mask[:, col_start[j]:col_start[j] + border_limit] = 1
                power_mask[row_end[-1] - border_limit:power_mask.shape[0], :] = 1
                power_mask[:, col_end[-1] - border_limit:power_mask.shape[1]] = 1

    # get IDs from labelled reference
    IDs_to_skip = np.unique(instances_true[power_mask==1])

    # get distribution of field sizes after segmentation
    unique_IDs, counts = np.unique(instances_true, return_counts=True)

    # exlcude fields that are too close to tile borders
    mask = ~np.isin(unique_IDs, IDs_to_skip)
    unique_IDs = unique_IDs[mask]
    counts = counts[mask]

    # exlude 0 (background) and super-small fields (~2 pixel) from sample
    pixelthresh = 0

    while True:
        mask = (unique_IDs != 0)  & (counts > pixelthresh)
        unique_IDs = unique_IDs[mask]
        counts = counts[mask]

        # get deciles and draw equally from them
        deciles = [perc for perc in range(10,100,10)]
        deciles_values = np.percentile(counts, deciles)
        decs = [0] + deciles_values.tolist() + [np.max(counts)]

        if len(decs) == len(set(decs)):
            df = pd.DataFrame({'decile_value': decs,
                   'excluded_pixel':pixelthresh})
            df.to_csv("decs_output.csv", index=False)
            break
        else:
            pixelthresh += 1

0 pixel excluded - [0, 1.0, 5.0, 49.0, 105.0, 205.0, 405.0, 856.0, 1995.0, 5671.0, np.int64(259902)]
1 pixel excluded - [0, 2.0, 4.0, 9.0, 30.0, 68.0, 136.0, 269.0, 541.0, 1231.0, np.int64(24316)]
0 pixel excluded - [0, 73.0, 110.0, 160.0, 237.0, 352.0, 544.0, 909.0999999999995, 1770.5999999999985, 4888.80000000001, np.int64(1386382)]
2 pixel excluded - [0, 3.0, 4.0, 7.0, 18.0, 49.0, 107.0, 223.0, 478.0, 1131.0, np.int64(24317)]
0 pixel excluded - [0, 1.0, 5.0, 49.0, 105.0, 205.0, 405.0, 856.0, 1995.0, 5671.0, np.int64(259902)]
1 pixel excluded - [0, 2.0, 4.0, 15.0, 39.0, 81.0, 157.0, 298.0, 584.0, 1293.0, np.int64(24149)]
0 pixel excluded - [0, 73.0, 110.0, 160.0, 237.0, 352.0, 544.0, 909.0999999999995, 1770.5999999999985, 4888.80000000001, np.int64(1386382)]
1 pixel excluded - [0, 2.0, 3.0, 8.0, 25.0, 61.0, 126.0, 253.0, 520.0, 1197.0, np.int64(24149)]


In [35]:
bin_ids = []
for ind in range(len(decs) -1):
    # get the unique_IDS of those fields, whose count (size) is within bin
    bin_ids.append(np.random.choice(unique_IDs[(counts > decs[ind]) & (counts <= decs[ind + 1])], int(sample_size/10), replace=False))



In [38]:
instances_true = np.where(np.isin(instances_true, np.concatenate(bin_ids)),
                        instances_true,
                        0)

In [39]:
np.unique(instances_true)

array([     0,      8,     18, ..., 398184, 398207, 398266],
      shape=(10001,), dtype=int32)

In [40]:
c = 0
t_exts = [i/100 for i in range(10,95,5)] 
t_bounds = [i/100 for i in range(10,95,5)]
for t_ext in t_exts:
    for t_bound in t_bounds:
        c += 1
print(c)

289


In [ ]:

bin_ids = []
for ind in range(len(decs) -1):
    # get the unique_IDS of those fields, whose count (size) is within bin
    bin_ids.append(np.random.choice(unique_IDs[(counts > decs[ind]) & (counts <= decs[ind + 1])], int(sample_size/10), replace=False))

instances_true = np.where(np.isin(instances_true, np.concatenate(bin_ids)),
                        instances_true,
                        0)


In [4]:
pathi = '/data/Aldhani/eoagritwin/fields/Auxiliary/grid_search/Brandenburg/2023/test1/unmasked_chips_256_20GSA-DE_BRB-2023_cropMask_lines_touch_true_crop_touch_true_linecrop_prediction_extent/intermediates/'


instances_pred = stackReader(checkPath(f'{pathi}0.4_0.2_instance_pred_10760_17982.tif'))
instances_true = stackReader(checkPath(f'{pathi}0.4_0.2_instance_true_10760_17982.tif'))


In [5]:
row_col_start = '10760_17982'

# --- 1) get predicted instance segmentation and relabel to 1..N (background 0) ---
instances_pred_raw = instances_pred

# Robust labeling: treat zeros as background; produce labels 0..P
binary_pred = instances_pred_raw != 0
instances_pred = measure.label(binary_pred, background=0)  # labels start at 1 for objects

# Flatten arrays once for a single-pass overlap computation
flat_true = instances_true.ravel()
flat_pred = instances_pred.ravel()

max_true_label = int(flat_true.max()) if flat_true.size else 0
max_pred_label = int(flat_pred.max()) if flat_pred.size else 0

# Build intersection matrix: rows=true-label, cols=pred-label
# intersection[i, j] = number of pixels where true==i and pred==j
# allocate (max_true_label+1, max_pred_label+1) to include the zero label row/col
intersection = np.zeros((max_true_label + 1, max_pred_label + 1), dtype=np.int32)

# Count only where both are non-zero (optional: we will still fill zeros row/col but it's faster to limit)
valid_mask = (flat_true > 0) & (flat_pred > 0)
if np.any(valid_mask):
    np.add.at(intersection, (flat_true[valid_mask], flat_pred[valid_mask]), 1)

# Compute per-label areas using bincount (includes zero index)
area_true = np.bincount(flat_true, minlength=(max_true_label + 1)).astype(np.int32)
area_pred = np.bincount(flat_pred, minlength=(max_pred_label + 1)).astype(np.int32)


In [8]:
area_true

array([7927120,       0,       0, ...,       0,       0,      21],
      shape=(228198,), dtype=int32)

In [ ]:

# Avoid division by zero: union = A_true + A_pred - intersection
# Vectorized IoU table (shape: (max_true_label+1, max_pred_label+1))
# We'll compute IoU only for labels >0 later; but it's fine to compute whole table.
union = (area_true[:, None] + area_pred[None, :] - intersection).astype(np.int32)
# protect against zero union by setting union to 1 where zero (IoU will be 0 because intersection is also 0)
zero_union_mask = union == 0
union_safe = union.copy()
union_safe[zero_union_mask] = 1
IoU_table = intersection / union_safe.astype(np.float32)
IoU_table[zero_union_mask] = 0.0  # ensure 0 where union was actually zero

# --- 3) prepare outputs in the same order the original iterated (np.unique order) ---
field_values = np.unique(instances_true)
# allocate lists
best_IoUs = []
centroid_rows = []
centroid_cols = []
centroid_IDs = []
centroid_IoUS = []
field_IDs = []
field_sizes = []
intersect_IDs = []
pred_field_overlap = []

# use regionprops for centroid & area extraction (map label -> region)
props_true = regionprops(instances_true)  # returns list of RegionProperties for labels > 0
# create mapping label -> RegionProperties for quick access
props_map = {p.label: p for p in props_true}

# we'll also need to get predicted label areas quickly (area_pred array already computed)
# and to access instances_pred by centroid coordinates

for fv in field_values:
    if fv == 0:
        continue

    field_IDs.append(int(fv))
    # area of the true field (from bincount)
    f_area = int(area_true[int(fv)])
    field_sizes.append(f_area)

    # centroid from regionprops (fallback if missing)
    if fv in props_map:
        centroid = props_map[fv].centroid  # (row, col) floats
        r = int(round(centroid[0]))
        c = int(round(centroid[1]))
    else:
        # fallback: compute centroid from coordinates (rare)
        coords = np.column_stack(np.where(instances_true == fv))
        if coords.size == 0:
            r, c = 0, 0
        else:
            mean_coords = np.mean(coords, axis=0)
            r, c = int(round(mean_coords[0])), int(round(mean_coords[1]))

    centroid_rows.append(int(r))
    centroid_cols.append(int(c))

    # find best predicted partner by IoU for this true label
    # Only consider pred labels >=1 .. max_pred_label
    if int(fv) <= max_true_label and max_pred_label >= 1:
        row_iou = IoU_table[int(fv), 1:(max_pred_label + 1)]  # exclude pred label 0
        if row_iou.size == 0 or np.all(row_iou == 0):
            best_j_rel = -1
            best_iou_val = 0.0
            best_pred_label = 0
        else:
            best_j_idx = np.argmax(row_iou)  # index in row_iou
            best_iou_val = float(row_iou[best_j_idx])
            best_pred_label = int(1 + best_j_idx)  # convert back to actual pred label
    else:
        best_pred_label = 0
        best_iou_val = 0.0

    best_IoUs.append(best_iou_val)
    intersect_IDs.append(int(best_pred_label))

    # compute predicted-field overlap fraction: intersection / pred_field_area
    if best_pred_label > 0:
        inter_area = int(intersection[int(fv), int(best_pred_label)])
        pred_area = int(area_pred[int(best_pred_label)]) if int(best_pred_label) <= max_pred_label else 0
        overlap_frac = (inter_area / pred_area) if pred_area > 0 else 0.0
    else:
        overlap_frac = 0.0
    pred_field_overlap.append(overlap_frac)

    # centroid-based predicted id and IoU
    # ensure centroid coordinates are inside bounds
    hr, hc = instances_pred.shape
    if 0 <= r < hr and 0 <= c < hc:
        pred_at_centroid = int(instances_pred[r, c])
    else:
        pred_at_centroid = 0

    centroid_IDs.append(pred_at_centroid)
    # centroid IoU: IoU_table[true_label, pred_at_centroid] if pred_at_centroid>0 else 0
    if pred_at_centroid > 0 and int(fv) <= max_true_label and pred_at_centroid <= max_pred_label:
        centroid_iou_val = float(IoU_table[int(fv), pred_at_centroid])
    else:
        centroid_iou_val = 0.0
    centroid_IoUS.append(centroid_iou_val)